In [ ]:
import os
import numpy as np
import json
import neuroglancer
from matplotlib import pyplot as plt
import tinybrain
from cloudvolume import CloudVolume
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

viewer = neuroglancer.Viewer()
print(viewer)

In [ ]:
color_filepath = os.path.join('../', 'neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}
    
surround = False
VOL_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(VOL_DIR)
volume_files = sorted([f for f in files if f.endswith('.npy') and surround == ('surround' in f) and 'test' not in f])
origin_files = sorted([f for f in files if f.endswith('.txt') and surround == ('surround' in f) and 'test' not in f])
    
structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    prefix = os.path.splitext(volume_filename)[0]
    structure = prefix.replace('atlasV7_10.0um_scoreVolume_', '').replace('_surround_200um', '')
    if structure not in origin_filename:
        print(structure, origin_filename)
        break
    
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    volume = np.load(os.path.join(VOL_DIR, volume_filename))
    origin = np.loadtxt(os.path.join(VOL_DIR, origin_filename))
    
    volume = np.rot90(volume, axes=(0,1))
    volume = np.flip(volume, axis=0)
    volume[volume > 0.8] = color
    volume = volume.astype(np.uint8)
    
    structure_volume_origin[structure] = (volume, origin)
print(structure_volume_origin.keys())

In [ ]:
x_length = 1000
y_length = 1000
z_length = 300
full_brain_volume_annotated = np.zeros((x_length, y_length, z_length), dtype=np.uint32)

for structure, (volume, origin) in structure_volume_origin.items():  
    
    x, y, z = origin
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = int(z) // 2 + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) // 2

    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    full_brain_volume_annotated[x_start:x_end, y_start:y_end,z_start:z_end] += volume


all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 20]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:
path = 'file:///net/birdstore/Active_Atlas_Data/data_root/pipeline_data/test'
factor = (2, 2, 1)
volumes = tinybrain.downsample_segmentation(full_brain_volume_annotated, factor=factor, num_mips=2, sparse=False)
volumes.insert(0, full_brain_volume_annotated)

In [ ]:
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint32', # Channel images might be 'uint8'
    encoding = 'compressed_segmentation', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [10000, 10000, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [512, 512, 16], # units are voxels
    volume_size = full_brain_volume_annotated.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(path, info=info, compress=False, progress=False)

for mip, volume in enumerate(volumes):
    vol.add_scale(np.array(factor) ** mip)
    vol.commit_info()
    vol = CloudVolume(path, mip=mip, compress=False, progress=False)
    vol[:, :, :] = volume[:, :, :]
    
vol.info['segment_properties'] = 'names'
vol.commit_info()

In [ ]:
VOL_DIR = '/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes'
atlas = 'atlasV8'
outfile =  os.path.join(VOL_DIR, atlas, 'full_brain_volume_annotated.npy')
np.save(outfile, full_brain_volume_annotated)

In [ ]:
cloudpath = f'file://{VOL_DIR}/{atlas}/mesh'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint32', # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [10000, 10000, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [512, 512, 16], # units are voxels
    volume_size = full_brain_volume_annotated.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False, progress=False)
vol.commit_info()
vol[:, :, :] = full_brain_volume_annotated[:, :, :]

In [ ]:
tq = LocalTaskQueue(parallel=2)
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False, progress=True)
tq.insert(tasks)
tasks = tc.create_mesh_manifest_tasks(cloudpath)
tq.insert(tasks)
tq.execute(progress=True)

In [ ]:
vol.viewer() # launches neuroglancer compatible web server on http://localhost:1337